# Priprava podatkov

Osnova projektne naloge bo [**Evidenca registriranih vozil**](https://podatki.gov.si/dataset/evidenca-registriranih-vozil-presek-stanja), ki vsebuje malo morje podatkov o vseh registriranih vozilih *(znamka, število prevoženih kilometrov, moč motorja, ...)* v določenem časovnem obdobju. Polega tega smo potrebovali še `obcine.csv` z imeni občin in `obcine.geojson` z mejami občin na zemljevidu.

## Knjižnice

In [1]:
import pandas as pd

## Branje podatkov

Podatki za leto 2022 so zaradi prevelike količine razdeljeni v šest `.csv` datotek. Za naš projekt so zadostovali spodnji **atributi registracije vozil**.

In [2]:
columns = {'B-Datum prve registracije vozila',
 'C13-Obcina uporabnika vozila (opis)',
 'C2-Starost lastnika vozila',
 'D1-Znamka',
 'D3-Komerc oznaka',
 'D42-Drzava (opis)',
 'P12-Nazivna moc',
 'P13-Vrsta goriva (opis)',
 'J-Kategorija in vrsta vozila (opis)',
 'Prevozeni kilometri milje'}

Z prebrali smo vseh 6 datotek, izločili nepotrebne stolpce in vrstice ter podatke shranili v `cars`.

In [3]:
files = ['data/raw_data/vozila2022_1.csv',
         'data/raw_data/vozila2022_2.csv',
         'data/raw_data/vozila2022_3.csv',
         'data/raw_data/vozila2022_4.csv',
         'data/raw_data/vozila2022_5.csv',
         'data/raw_data/vozila2022_6.csv']

for f in files:
    # izbira stolpcev (atributi)
    data = pd.read_csv(f, sep=';', encoding='dbcs', usecols=columns, low_memory=False, on_bad_lines='skip')
    # izbira vrstic (avtomobili)
    data = data[data['J-Kategorija in vrsta vozila (opis)'] == 'osebni avtomobil']
    cars = pd.concat([data])
%store cars



#podatki o populaciji Slovenije https://www.macrotrends.net/countries/SVN/slovenia/population
population = pd.read_csv('data/slovenia-population1950_2022.csv',sep=',',encoding='utf-8', on_bad_lines='skip')
%store population



Stored 'cars' (DataFrame)
Stored 'population' (DataFrame)


In [4]:
files = ['data/raw_data/vozila2022_1.csv',
         'data/raw_data/vozila2022_2.csv',
         'data/raw_data/vozila2022_3.csv',
         'data/raw_data/vozila2022_4.csv',
         'data/raw_data/vozila2022_5.csv',
         'data/raw_data/vozila2022_6.csv']

for f in files:
    # izbira stolpcev (atributi)
    data = pd.read_csv(f, sep=';', encoding='dbcs', usecols=columns, low_memory=False, on_bad_lines='skip')
    # izbira vrstic (avtomobili)
    data = data[data['J-Kategorija in vrsta vozila (opis)'] == 'osebni avtomobil']
    cars = pd.concat([data])
%store cars



#podatki o populaciji Slovenije https://www.macrotrends.net/countries/SVN/slovenia/population
population = pd.read_csv('data/slovenia-population1950_2022.csv',sep=',',encoding='utf-8', on_bad_lines='skip')
%store population



Stored 'cars' (DataFrame)
Stored 'population' (DataFrame)


## Popravljanje podatkov

Za uspešno vizualizacijo z zemeljevidom, razdeljenim na občine, smo bili primorani popraviti imena nekaterih občin. V surovih podatkih je črko `č` nadomeščal `c`, hkrati pa je bilo poimenovanje štirih občin napačno. Težavo smo rešili s **slovarjem preslikav**, kjer ključ predstavlja preslikavo na pravilno ime občine. 

In [5]:
wrong_comm_names = set(cars["C13-Obcina uporabnika vozila (opis)"])

correct_comm_names = pd.read_csv('data/obcine.csv', usecols=['OB_UIME'], low_memory=False)['OB_UIME'].tolist()

comm_names_links = dict.fromkeys(wrong_comm_names)

for c_name in correct_comm_names:
    for w_name in wrong_comm_names:
        
        if c_name.lower().replace('c', '').replace('č', '') \
            == w_name.lower().replace('c', '').replace('č', ''):
                
            comm_names_links[w_name] = c_name
            break
            
comm_names_links['Sveti Jurij'] = 'Sveti Jurij ob Ščavnici'
comm_names_links['Šentjur pri Celju'] = 'Šentjur'
comm_names_links['Ravne'] = 'Ravne na Koroškem'
comm_names_links['Sveti Andraž v Slov.goricah'] = 'Sveti Andraž v Slov. goricah'

Podatke smo shranili v slovar `comm_data`, kje kot vrednost hranimo podatke posamezne občine oziroma ključa. Slednji slovar je deljena spremenljivka med ostalimi **jupyter zvezki**.

In [6]:
comm_data = {comm_names_links[w_name]:cars[cars['C13-Obcina uporabnika vozila (opis)'] == w_name] for w_name in wrong_comm_names}
del comm_data[None]
%store comm_data

Stored 'comm_data' (dict)


{'Videm':        B-Datum prve registracije vozila C13-Obcina uporabnika vozila (opis)  \
 270                           6.01.2000                               Videm   
 888                          13.02.2015                               Videm   
 894                           7.05.2019                               Videm   
 1081                         18.09.2003                               Videm   
 2555                         11.02.2013                               Videm   
 ...                                 ...                                 ...   
 473669                        6.06.2019                               Videm   
 473713                       19.09.2013                               Videm   
 474722                       17.11.2009                               Videm   
 475115                        4.11.2021                               Videm   
 475680                       26.02.2018                               Videm   
 
         C2-Starost lastnika 